In [ ]:
%cd ../airflow
%pwd

In [ ]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2019")
extractor.extract_dynamic_files("2019")

In [ ]:
import pandas as pd
import uuid

In [ ]:
partidas_df = pd.read_csv("./data/raw/2019/partidas/1.csv")

partidas_df.rename(
    columns={
        "home_team": "clubeMandanteID",
        "away_team": "clubeVisitanteID",
        "home_score": "mandantePlacar",
        "away_score": "visitantePlacar",
        "round": "rodada",
    },
    inplace=True,
)

def get_resultado(row):
    if row["mandantePlacar"] > row["visitantePlacar"]:
        return "Casa"
    elif row["mandantePlacar"] < row["visitantePlacar"]:
        return "Visitante"
    else:
        return "Empate"
    
partidas_df["resultado"] = partidas_df.apply(get_resultado, axis=1)
partidas_df["partidaID"] = [uuid.uuid4().hex for _ in range(len(partidas_df))]
partidas_df["temporada"] = 2019

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

In [ ]:
from glob import glob

file_list = glob("./data/raw/2019/rodada/*.csv")
scouts_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    scouts_df = pd.concat([scouts_df, tmp_df], sort=True)

scouts_df = scouts_df.rename(
    columns={
        "atletas.clube_id": "clubeID",
        "atletas.atleta_id": "atletaID",
        "atletas.rodada_id": "rodada",
        "atletas.pontos_num": "pontos",
        "atletas.media_num": "pontosMedia",
        "atletas.preco_num": "preco", 
        "atletas.variacao_num": "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

In [ ]:
posicoes_df = pd.read_csv("./data/raw/2019/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

In [ ]:
from glob import glob

file_list = glob("./data/raw/2019/rodada/*.csv")
atletas_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    atletas_df = pd.concat([atletas_df, tmp_df], sort=True)

atletas_df.rename(
    columns={
        "atletas.clube_id": "clubeID",
        "atletas.atleta_id": "atletaID",
        "atletas.apelido": "apelido",
    },
    inplace=True,
)

atletas_df = atletas_df.merge(
    right=posicoes_df,
    left_on="atletas.posicao_id",
    right_on="abreviacao",
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df.drop_duplicates(inplace=True)
atletas_df.reset_index(drop=True, inplace=True)

atletas_df

In [ ]:
clubes_df = pd.read_csv("./data/raw/2019/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df